In [17]:
import requests
from bs4 import BeautifulSoup
from akshare.utils import demjson
from akshare.utils.tqdm import get_tqdm
import pandas as pd
from datetime import datetime, timedelta
from akshare.tool.trade_date_hist import tool_trade_date_hist_sina

# ===== 1. 数据预处理 ===== 
import numpy as np
import torch
import torch.nn as nn
from torch  import amp
from torch.utils.data  import Dataset, DataLoader
from sklearn.preprocessing  import MinMaxScaler
import joblib
import os
# import py_mini_racer

In [18]:
# def is_trading_day():
#     today = datetime.now().strftime('%Y-%m-%d') 
#     # 获取交易日历（新浪财经）
#     trade_date_df = tool_trade_date_hist_sina() 
#     # 判断今天是否在交易日历中
#     return today in trade_date_df['trade_date'].values

# if is_trading_day():
#     print("今天是交易日")
# else:
#     print("今天不是交易日")

# is_trading_day = is_trading_day()
# today = datetime.now().date()
trade_date_df = tool_trade_date_hist_sina() 
 
# 确保日期列是字符串格式
trade_date_list = trade_date_df['trade_date'].astype(str).tolist()
 
# 获取当前日期
today = datetime.now().date() 
lastday = today - timedelta(days=1)
# 找到最近交易日
recent_trade_date = today 
while str(recent_trade_date) not in trade_date_list:
    recent_trade_date -= timedelta(days=1)
    lastday -= timedelta(days=1)
while str(lastday) not in trade_date_list:
    lastday -= timedelta(days=1)
 
print(f"最近交易日：{recent_trade_date}")

最近交易日：2025-09-15


In [19]:
print(type(recent_trade_date))

<class 'datetime.date'>


In [26]:
start_year = "2025"
current_year = datetime.now().year

# js_code = py_mini_racer.MiniRacer()
# setting_file_path = "./ths.js"
# with open(setting_file_path, encoding="utf-8") as f:
#     js_content = f.read()
# js_code.eval(js_content)
# v_code = js_code.call("v")

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/89.0.4389.90 Safari/537.36",
        # "Cookie": f"v={v_code}",
    }
headers_bk = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/89.0.4389.90 Safari/537.36",
            "Referer": "https://q.10jqka.com.cn",
            "Host": "d.10jqka.com.cn",
            # "Cookie": f"v={v_code}",
        }

link_code_ls = []
bk_data_ls = []
code_ls = []

for suffix in ["gn/","thshy/"]:
    url = f"https://q.10jqka.com.cn/{suffix}"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, features="lxml")
    divs = soup.find_all(name="div", attrs={"class": "cate_items"})
    for div in divs:
        # 在每个div中查找所有的a标签
        links = div.find_all('a') 
        for link in links:
            # 获取href属性
            href = link.get('href') 
            # 获取文本内容
            text = link.text 
            link_code_ls.append((href,  text))

print(len(link_code_ls))
for symbol_url, name in link_code_ls:
    try:
        r = requests.get(symbol_url, headers=headers)
        soup_today = BeautifulSoup(r.text, features="lxml")
        symbol_code = soup_today.find(name="div", attrs={"class": "board-hq"}).find("span").text
        big_df = pd.DataFrame()
        temp_start_year = start_year
        
        if os.path.exists(f"./data/{symbol_code}.csv"):
            big_df = pd.read_csv(f"./data/{symbol_code}.csv",  encoding="utf-8")
            if not big_df.empty:
                big_df["date"] = pd.to_datetime(big_df["date"], errors="coerce").dt.date
                if big_df["date"].iloc[-1] >= recent_trade_date:
                    bk_data_ls.append((name, symbol_code, big_df))
                    code_ls.append(symbol_code)
                    continue
                if big_df['date'].iloc[-1].year > int(temp_start_year):
                    temp_start_year = str(big_df["date"].iloc[-1].year)
                    idx = big_df.index[big_df['date'].dt.year == int(temp_start_year)].tolist()
                    first_idx = idx[0] if idx else None
                    if first_idx is not None:
                        big_df = big_df.iloc[:first_idx] 
                elif (big_df['date'].iloc[-1] == lastday):
                    dd_texts = [dd.text for dd in soup_today.find_all("dd")] 
                    close = soup_today.select_one('span.board-xj.arr-rise,  span.board-xj.arr-fall').text
                    open, low, high, volume, amount = dd_texts[0], dd_texts[2], dd_texts[3], dd_texts[4], dd_texts[9]
                    new_list = [
                        recent_trade_date,
                        float(symbol_code),
                        float(open),
                        float(high),
                        float(low),
                        float(close),
                        float(volume.replace(",", "")),
                        float(amount.replace(",", "")),
                    ]
                    new_row = pd.DataFrame([new_list], columns=big_df.columns) 
                    big_df = pd.concat([big_df,  new_row], ignore_index=True)
                    bk_data_ls.append((name, symbol_code, big_df))
                    code_ls.append(symbol_code)
                    big_df.to_csv(f"./data/{symbol_code}.csv", index=False, encoding='utf-8-sig')
                    continue

        tqdm = get_tqdm()
        for year in tqdm(range(int(temp_start_year), current_year + 1), leave=False):
            url = f"http://d.10jqka.com.cn/v4/line/bk_{symbol_code}/01/{year}.js"
            r = requests.get(url, headers=headers_bk)
            data_text = r.text
            try:
                demjson.decode(data_text[data_text.find("{") : -1])
                temp_df = demjson.decode(data_text[data_text.find("{") : -1])
                temp_df = pd.DataFrame(temp_df["data"].split(";"))
                temp_df = temp_df.iloc[:, 0].str.split(",", expand=True)
                big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)
            except Exception as e:  # noqa: E722
                print(e, "decode fail:", url, "text as:", data_text)
                break
        big_df = big_df.iloc[:,  :7]
        big_df.columns  = ["date", "open", "high", "low", "close", "volume", "amount"]
        big_df.insert(1,  'code', symbol_code)
        big_df["date"] = pd.to_datetime(big_df["date"], errors="coerce").dt.date
        big_df["code"] = pd.to_numeric(big_df["code"], errors="coerce")
        big_df["open"] = pd.to_numeric(big_df["open"], errors="coerce")
        big_df["high"] = pd.to_numeric(big_df["high"], errors="coerce")
        big_df["low"] = pd.to_numeric(big_df["low"], errors="coerce")
        big_df["close"] = pd.to_numeric(big_df["close"], errors="coerce")
        big_df["volume"] = pd.to_numeric(big_df["volume"], errors="coerce")
        big_df["amount"] = pd.to_numeric(big_df["amount"], errors="coerce")
        

        if big_df["date"].iloc[-1] != recent_trade_date:
            dd_texts = [dd.text for dd in soup_today.find_all("dd")] 
            close = soup_today.select_one('span.board-xj.arr-rise,  span.board-xj.arr-fall').text
            open, low, high, volume, amount = dd_texts[0], dd_texts[2], dd_texts[3], dd_texts[4], dd_texts[9]
            new_list = [
                recent_trade_date,
                float(symbol_code),
                float(open),
                float(high),
                float(low),
                float(close),
                float(volume.replace(",", "")),
                float(amount.replace(",", "")),
            ]
            new_row = pd.DataFrame([new_list], columns=big_df.columns) 
            big_df = pd.concat([big_df,  new_row], ignore_index=True)
        bk_data_ls.append((name, symbol_code, big_df))
        code_ls.append(symbol_code)
        big_df.to_csv(f"./data/{symbol_code}.csv", index=False, encoding='utf-8-sig')
    except Exception as e:
        print(e, " wrong data:", f"https://d.10jqka.com.cn/v4/line/bk_{symbol_code}/01/{year}.js", symbol_url, big_df.head())
        break


442
'NoneType' object has no attribute 'find'  wrong data: https://d.10jqka.com.cn/v4/line/bk_881156/01/2025.js http://q.10jqka.com.cn/thshy/detail/code/881138/          date      code      open      high       low     close       volume  \
0  2025-01-02  881156.0  2221.520  2224.233  2091.513  2110.404  479628840.0   
1  2025-01-03  881156.0  2108.265  2111.887  2039.321  2043.119  405636270.0   
2  2025-01-06  881156.0  2039.098  2043.666  1999.205  2008.669  348719560.0   
3  2025-01-07  881156.0  2010.410  2021.026  1990.481  2019.256  273896540.0   
4  2025-01-08  881156.0  2018.007  2041.064  1983.092  2026.934  330434890.0   

         amount  
0  1.006006e+10  
1  6.526986e+09  
2  5.031152e+09  
3  4.438087e+09  
4  5.483813e+09  


In [21]:
print(bk_data_ls[0][0])
print(bk_data_ls[0][1])
print(bk_data_ls[0][2].tail())

阿尔茨海默概念
886056
           date      code      open      high       low     close  \
168  2025-09-10  886056.0  1325.734  1339.842  1319.055  1327.358   
169  2025-09-11  886056.0  1315.080  1340.425  1287.400  1340.425   
170  2025-09-12  886056.0  1342.274  1359.268  1337.076  1350.272   
171  2025-09-12  886056.0  1342.270  1359.270  1337.080  1350.270   
172  2025-09-15  886056.0  1349.380  1359.670  1346.040  1355.560   

           volume        amount  
168  5.103168e+08  9.774379e+09  
169  7.672469e+08  1.480606e+10  
170  7.405859e+08  1.411970e+10  
171  7.405900e+02  1.412000e+02  
172  6.558900e+02  1.272300e+02  


In [22]:
def save_Scalers(scalers, filepath):
    """保存所有股票的Scaler到文件"""
    joblib.dump(scalers, filepath)
def load_Scalers(filepath):
    if os.path.exists(filepath):
        return joblib.load(filepath)
    else:
        return {}

In [23]:
code_scaler_path = "./model/code_scaler.save"
if os.path.exists(code_scaler_path):
    code_scaler = joblib.load(code_scaler_path)
    if len(code_ls) != len(bk_data_ls):
        code_df = pd.DataFrame(code_ls, columns=['code'])
        code_scaler.partial_fit(code_df)
else:
    code_df = pd.DataFrame(code_ls, columns=['code'])
    code_scaler = MinMaxScaler(feature_range=(0, 1))
    code_scaler.fit(code_df)
    save_Scalers(code_scaler, code_scaler_path)

In [24]:
class StockDataset(Dataset):
    def __init__(self, bk_data_ls, seq_length=16, forecast_gap=2, scaler_filepath='./model/scalers.sav'):
        """
        bk_data_ls: List of tuples (code, data), where:
            - code: 股票代码（str）
            - data: DataFrame，列顺序为 [open, high, low, close, volume, amount]
        seq_length_short: 短期序列长度（可选）
        seq_length: 长期依赖序列长度（用于输入）
        forecast_gap: 预测几天后的收盘价，如后天 = 2
        """
        self.seq_length  = seq_length
        self.forecast_gap  = forecast_gap
        self.scalers  = load_Scalers(scaler_filepath) # 存储每个股票的归一化器
 
        all_X = []
        all_y = []

        for _name, code, data in bk_data_ls:
            data = data[["open", "high", "low", "close", "volume", "amount"]]
            if code in self.scalers:
                scaler = self.scalers[code]
                scaled_data = scaler.partial_fit(data).transform(data) 
                self.scalers[code]  = scaler  # 保存 scaler 供后续使用 
            else:
                scaler = MinMaxScaler(feature_range=(0, 1))
                scaled_data = scaler.fit_transform(data) 
                self.scalers[code]  = scaler  # 保存 scaler 供后续使用 
    
            # 构造样本
            X, y = [], []
            max_index = len(data) - seq_length - forecast_gap
            for i in range(max_index):
                # 输入序列：seq_length 天的特征
                seq_features = scaled_data[i:i + seq_length]
                code_feature = code_scaler.transform(data[['code']].iloc[i:i + seq_length])
                seq_features = np.insert(seq_features,  1, code_feature, axis=1) 
                # 标签：forecast_gap 天后的收盘价（第3列）
                target_idx = i + seq_length + forecast_gap - 1
                target_close = scaled_data[target_idx, 3]
 
                X.append(seq_features) 
                y.append(target_close) 
 
            # 转换为 numpy 并保存 
            all_X.extend(X) 
            all_y.extend(y) 
 
        # 统一转换为 tensor
        self.X = torch.tensor(np.array(all_X),  dtype=torch.float16) 
        self.y = torch.tensor(np.array(all_y),  dtype=torch.float16).view(-1,  1)
        save_Scalers(self.scalers, scaler_filepath)  # 保存所有股票的Scaler

    def __len__(self):
        return len(self.X)
 
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
 
    def get_scaler(self, code):
        return self.scalers.get(code,  None)
    def inverse_transform(self, code, scaled_values):
        """逆归一化"""
        scaler = self.get_scaler(code)
        if scaler:
            dummy = np.zeros((len(scaled_values),  self.X.shape[2]))
            dummy[:, 3] = scaled_values  # 将预测值放入close列
            return scaler.inverse_transform(dummy)[:, 3]
        else:
            raise ValueError(f"No scaler found for code: {code}")

    

class MultiScaleAttentionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size = 64, num_layers_long = 3, num_layers_short = 2):
        super().__init__()
        
        self.lstm_long  = nn.LSTM(input_size, hidden_size, num_layers=num_layers_long, batch_first=True, dropout=0.2 if num_layers_long > 1 else 0)
        self.lstm_short  = nn.LSTM(input_size, hidden_size, num_layers=num_layers_short, batch_first=True, dropout=0.2 if num_layers_short > 1 else 0)
        self.attn  = nn.MultiheadAttention(hidden_size, num_heads=4, batch_first=True, )  # 4头注意力 
        
        self.regressor  = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.Linear(32, 1)
        )
 
    def forward(self, x1, seq_length_short=7):
        out1, _ = self.lstm_long(x1) 
        out2, _ = self.lstm_short(x1[:, -seq_length_short:, :]) # lstm_out: [batch, seq_len, hidden]
        lstm_out = torch.cat((out1,  out2), dim=1)
        attn_out, _ = self.attn(lstm_out,  lstm_out, lstm_out, need_weights=False) 
        out = self.regressor(attn_out[:, -1, :]) 
        return out

In [25]:
# 创建数据集 
dataset = StockDataset(bk_data_ls, seq_length=16, forecast_gap=2)
train_size = int(0.8 * len(dataset))
train_set, test_set = torch.utils.data.random_split( 
    dataset, [train_size, len(dataset) - train_size]
)

# 数据加载器 
train_loader = DataLoader(train_set, batch_size=100000, shuffle=True)
test_loader = DataLoader(test_set, batch_size=100000)

# 模型初始化
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 加载模型（需先实例化结构）
model = MultiScaleAttentionLSTM(
    input_size=7,  # 7个特征 
    hidden_size=64,
    num_layers_long = 3, 
    num_layers_short = 2
).to(device)

if os.path.exists('./model/model.pth'):
    model.load_state_dict(torch.load('./model/model.pth')) 

# 训练参数 
criterion = nn.MSELoss()
scaler = amp.GradScaler(device)
optimizer = torch.optim.Adam(model.parameters(),  lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
    optimizer, 
    mode='min',       # 监控验证损失
    factor=0.5,       # 学习率衰减系数 
    patience=5,       # 容忍5个epoch无改善
)

# ===== 4. 训练循环 =====
for epoch in range(100):
    model.train() 
    train_loss = 0 
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device),  y_batch.to(device) 
        
        with amp.autocast(device_type=device): 
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
        train_loss += loss.item()
        # 反向传播（自动处理精度）
        scaler.scale(loss).backward()   # 缩放梯度
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)          # 更新参数 
        scaler.update()                 # 调整缩放因子 
        optimizer.zero_grad() 
    print(f"Epoch {epoch+1}: Current LR = {scheduler.get_last_lr()[0]}") 
    # 验证
    model.eval() 
    test_loss = 0
    with torch.no_grad(): 
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device),  y_test.to(device) 
            preds = model(X_test)
            test_loss += criterion(preds, y_test).item()
    
    scheduler.step(test_loss) 
    print(f'Epoch {epoch} | Train Loss: {train_loss/len(train_loader):.6f} | Test Loss: {test_loss/len(test_loader):.6f}')

torch.save(model.state_dict(),  './model/model.pth') 

# ===== 5. 预测示例 =====
model.eval() 
with torch.no_grad():
    for name, code, data in bk_data_ls:
        scaler = dataset.get_scaler(code)
        max_index = len(data) - dataset.seq_length
        seq_features = scaler.transform(data[["open", "high", "low", "close", "volume", "amount"]].iloc[max_index:])
        code_feature = code_scaler.transform(data[['code']].iloc[max_index:])
        seq_features = np.insert(seq_features,  1, code_feature, axis=1)
        X = torch.tensor(seq_features,  dtype=torch.float16).to(device)
        X=X.unsqueeze(0)  # 添加批次维度

        prediction = model(X)
        scaled_pred = prediction.cpu().numpy() 
        
        # 逆归一化收盘价 
        dummy = np.zeros((1,  6))
        dummy[:, 3] = scaled_pred  # 将预测值放入close列
        real_pred = scaler.inverse_transform(dummy)[0,  3]
        if real_pred > data.iloc[-1,  3]:
            print("high")
        else:
            print("low")
        print(f'stock name: {name}, stock code: {code}, 今天收盘价：{data.iloc[-1,  3]}, 预测的后天收盘价: {real_pred:.2f}')



KeyError: "None of [Index(['code'], dtype='object')] are in the [columns]"